In [1]:
import numpy as np
import pandas as pd
import plotly
#import iFeatureOmegaCLI
import Bio
import modlamp

In [2]:
protien = pd.read_csv("./sequences_training.txt", header=None)
protien.columns = ["protien_seq", "class"]
protien.head()

,protien_seq,class
0,MLKQVEIFTDGSCLGNPGPGGYGAILRYRGREKTFSAGYTRTTNNR...,nonDRNA
1,MEQKKMKYLENLVGKTPMLELIFDYKGEERRIFVKNESYNLTGSIK...,nonDRNA
2,MTILFQLALAALVILSFVMVIGVPVAYASPQDWDRSKQLIFLGSGL...,nonDRNA
3,MSKIERISAFLNDKEVDMTFITNPTTLNYLTGLAIDPHERIAGLMI...,nonDRNA
4,MSDQQQPPVYKIALGIEYDGSKYYGWQRQNEVRSVQEKLEKALSQV...,RNA


In [12]:
temp = protien.head()
#Bio.SeqUtils.ProtParam.ProteinAnalysis(temp)
Bio.__version__

'1.86'

Getting features based on Physicochemical and Structural Features using Bio

In [12]:
from Bio.SeqUtils.ProtParam import ProteinAnalysis

In [14]:
temp2 = ProteinAnalysis(temp["protien_seq"][0])

In [15]:
#print(temp2.aromaticity())
print(temp2.molecular_weight())
print(temp2.isoelectric_point())
#print(temp2.charge_at_pH())
print(temp2.gravy())
print(temp2.secondary_structure_fraction())
#print(temp2.count_amino_acids())
print(temp2.amino_acids_percent)

17596.7878
8.42854824066162
-0.6309677419354839
(0.3419354838709678, 0.23870967741935487, 0.32903225806451614)
{'A': 9.03225806451613, 'C': 1.935483870967742, 'D': 4.516129032258065, 'E': 7.741935483870968, 'F': 1.2903225806451613, 'G': 9.03225806451613, 'H': 3.225806451612903, 'I': 4.516129032258065, 'K': 7.096774193548387, 'L': 7.741935483870968, 'M': 2.5806451612903225, 'N': 4.516129032258065, 'P': 3.225806451612903, 'Q': 5.161290322580645, 'R': 6.451612903225806, 'S': 2.5806451612903225, 'T': 6.451612903225806, 'V': 5.806451612903226, 'W': 3.870967741935484, 'Y': 3.225806451612903}


In [16]:
temp2.amino_acids_percent.keys()

dict_keys(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y'])

Positive-to-Negative Residue Ratio:
$\frac{K+R+H}{D+E}$
This formula has the potential to lead to a division by 0, so I will be adding a very small number $\epsilon$:
$\frac{K+R+H}{D+E+\epsilon}$

In [17]:
temp3 = temp2.count_amino_acids()
print(temp3['K'] + temp3['R'] + temp3['H'] / temp3['D'] + temp3['E'])

33.714285714285715


Amino Acid Categories [Source](https://www.britannica.com/science/amino-acid/Standard-amino-acids)
- Type 1 Nonpolar:
  - A, G, I, L, M, F, P, W, V
- Type 2 Polar, Uncharged:
  - S, C, N, Q, T, Y
- Type 3 Acidic:
  - D, E
- Type 4 Basic:
  - K, R, H

*note: we could also try combining types 2, 3 and 4 into 1 category if needed*

Features I want to start with:
- molecular weight
- isoelectric point
- gravy score
- all 3 secondary structures
- all 20 amino acid percenages
- protien length
- positive-to-negative residue ratio
- all 4 amino acid category counts

In [18]:
# the mini dataframe I will be using to figure out how to extract my features on interest
temp

,protien_seq,class
0,MLKQVEIFTDGSCLGNPGPGGYGAILRYRGREKTFSAGYTRTTNNR...,nonDRNA
1,MEQKKMKYLENLVGKTPMLELIFDYKGEERRIFVKNESYNLTGSIK...,nonDRNA
2,MTILFQLALAALVILSFVMVIGVPVAYASPQDWDRSKQLIFLGSGL...,nonDRNA
3,MSKIERISAFLNDKEVDMTFITNPTTLNYLTGLAIDPHERIAGLMI...,nonDRNA
4,MSDQQQPPVYKIALGIEYDGSKYYGWQRQNEVRSVQEKLEKALSQV...,RNA


In [19]:
temp = temp.copy(deep=True)
temp["protien_an"] = temp["protien_seq"].apply(ProteinAnalysis)
temp

,protien_seq,class,protien_an
0,MLKQVEIFTDGSCLGNPGPGGYGAILRYRGREKTFSAGYTRTTNNR...,nonDRNA,<Bio.SeqUtils.ProtParam.ProteinAnalysis object...
1,MEQKKMKYLENLVGKTPMLELIFDYKGEERRIFVKNESYNLTGSIK...,nonDRNA,<Bio.SeqUtils.ProtParam.ProteinAnalysis object...
2,MTILFQLALAALVILSFVMVIGVPVAYASPQDWDRSKQLIFLGSGL...,nonDRNA,<Bio.SeqUtils.ProtParam.ProteinAnalysis object...
3,MSKIERISAFLNDKEVDMTFITNPTTLNYLTGLAIDPHERIAGLMI...,nonDRNA,<Bio.SeqUtils.ProtParam.ProteinAnalysis object...
4,MSDQQQPPVYKIALGIEYDGSKYYGWQRQNEVRSVQEKLEKALSQV...,RNA,<Bio.SeqUtils.ProtParam.ProteinAnalysis object...


In [20]:
# repeat this process for the isoelectric point and gravy score functions
temp["mol_weighht"] = temp["protien_an"].apply(lambda x: x.molecular_weight())
temp

,protien_seq,class,protien_an,mol_weighht
0,MLKQVEIFTDGSCLGNPGPGGYGAILRYRGREKTFSAGYTRTTNNR...,nonDRNA,<Bio.SeqUtils.ProtParam.ProteinAnalysis object...,17596.7878
1,MEQKKMKYLENLVGKTPMLELIFDYKGEERRIFVKNESYNLTGSIK...,nonDRNA,<Bio.SeqUtils.ProtParam.ProteinAnalysis object...,37260.7955
2,MTILFQLALAALVILSFVMVIGVPVAYASPQDWDRSKQLIFLGSGL...,nonDRNA,<Bio.SeqUtils.ProtParam.ProteinAnalysis object...,6764.1710
3,MSKIERISAFLNDKEVDMTFITNPTTLNYLTGLAIDPHERIAGLMI...,nonDRNA,<Bio.SeqUtils.ProtParam.ProteinAnalysis object...,39968.9808
4,MSDQQQPPVYKIALGIEYDGSKYYGWQRQNEVRSVQEKLEKALSQV...,RNA,<Bio.SeqUtils.ProtParam.ProteinAnalysis object...,30399.2510


In [21]:
temp_struc = pd.DataFrame(list(temp["protien_an"].apply(lambda x: x.secondary_structure_fraction())), columns=["Helix", "Turn", "Sheet"])
temp_struc

,Helix,Turn,Sheet
0,0.341935,0.238710,0.329032
1,0.366071,0.303571,0.366071
2,0.322581,0.209677,0.580645
3,0.334254,0.281768,0.356354
4,0.300000,0.248148,0.355556


In [22]:
temp2 = temp["protien_an"].apply(lambda x: x.count_amino_acids()).apply(pd.Series)
temp2

,A,C,D,E,F,G,H,I,K,L,M,N,P,Q,R,S,T,V,W,Y
0,14,3,7,12,2,14,5,7,11,12,4,7,5,8,10,4,10,9,6,5
1,18,2,18,25,14,30,3,29,37,31,12,20,11,4,10,23,14,22,1,12
2,6,0,2,0,5,4,0,5,1,11,2,1,2,3,1,4,1,11,2,1
3,31,4,26,31,19,28,11,30,21,25,13,16,15,8,12,17,20,25,1,9
4,31,3,15,13,9,16,11,10,11,20,6,11,15,12,20,10,14,26,5,12


In [23]:
pd.concat([temp, temp2], axis=1)

,protien_seq,class,protien_an,mol_weighht,A,C,D,E,F,G,...,M,N,P,Q,R,S,T,V,W,Y
0,MLKQVEIFTDGSCLGNPGPGGYGAILRYRGREKTFSAGYTRTTNNR...,nonDRNA,<Bio.SeqUtils.ProtParam.ProteinAnalysis object...,17596.7878,14,3,7,12,2,14,...,4,7,5,8,10,4,10,9,6,5
1,MEQKKMKYLENLVGKTPMLELIFDYKGEERRIFVKNESYNLTGSIK...,nonDRNA,<Bio.SeqUtils.ProtParam.ProteinAnalysis object...,37260.7955,18,2,18,25,14,30,...,12,20,11,4,10,23,14,22,1,12
2,MTILFQLALAALVILSFVMVIGVPVAYASPQDWDRSKQLIFLGSGL...,nonDRNA,<Bio.SeqUtils.ProtParam.ProteinAnalysis object...,6764.1710,6,0,2,0,5,4,...,2,1,2,3,1,4,1,11,2,1
3,MSKIERISAFLNDKEVDMTFITNPTTLNYLTGLAIDPHERIAGLMI...,nonDRNA,<Bio.SeqUtils.ProtParam.ProteinAnalysis object...,39968.9808,31,4,26,31,19,28,...,13,16,15,8,12,17,20,25,1,9
4,MSDQQQPPVYKIALGIEYDGSKYYGWQRQNEVRSVQEKLEKALSQV...,RNA,<Bio.SeqUtils.ProtParam.ProteinAnalysis object...,30399.2510,31,3,15,13,9,16,...,6,11,15,12,20,10,14,26,5,12


In [27]:
temp2["work"] = temp2["D"] + temp2["E"]
temp2

,A,C,D,E,F,G,H,I,K,L,...,N,P,Q,R,S,T,V,W,Y,work
0,14,3,7,12,2,14,5,7,11,12,...,7,5,8,10,4,10,9,6,5,19
1,18,2,18,25,14,30,3,29,37,31,...,20,11,4,10,23,14,22,1,12,43
2,6,0,2,0,5,4,0,5,1,11,...,1,2,3,1,4,1,11,2,1,2
3,31,4,26,31,19,28,11,30,21,25,...,16,15,8,12,17,20,25,1,9,57
4,31,3,15,13,9,16,11,10,11,20,...,11,15,12,20,10,14,26,5,12,28


In [25]:
temp2

,A,C,D,E,F,G,H,I,K,L,M,N,P,Q,R,S,T,V,W,Y
0,14,3,7,12,2,14,5,7,11,12,4,7,5,8,10,4,10,9,6,5
1,18,2,18,25,14,30,3,29,37,31,12,20,11,4,10,23,14,22,1,12
2,6,0,2,0,5,4,0,5,1,11,2,1,2,3,1,4,1,11,2,1
3,31,4,26,31,19,28,11,30,21,25,13,16,15,8,12,17,20,25,1,9
4,31,3,15,13,9,16,11,10,11,20,6,11,15,12,20,10,14,26,5,12


# Final Code for Feature Extraction
The clean_data function removes unkown AA by default or it can replace unknown AA with the following mappings:
- B -> D
- Z -> E
- J -> L
- U -> C
- O -> K
- X -> G

*note: X represents an unknown AA, I am replacing it with G because it is considered a conservative estimate?*

In [3]:
from FeatureExtraction import extract_features, clean_data

In [4]:
clean_protiens = clean_data(protien)

In [5]:
bio_features = extract_features(clean_protiens)

In [6]:
bio_features

,class,protien_len,mol_weighht,isoelectric_point,Helix,Turn,Sheet,type1,type2,type3,...,N,P,Q,R,S,T,V,W,Y,pos_neg_rat
0,nonDRNA,155,17596.7878,-0.630968,0.341935,0.238710,0.329032,73,37,19,...,4.516129,3.225806,5.161290,6.451613,2.580645,6.451613,5.806452,3.870968,3.225806,1.368420
1,nonDRNA,336,37260.7955,-0.201190,0.366071,0.303571,0.366071,168,75,43,...,5.952381,3.273810,1.190476,2.976190,6.845238,4.166667,6.547619,0.297619,3.571429,1.162790
2,nonDRNA,62,6764.1710,1.579032,0.322581,0.209677,0.580645,48,10,2,...,1.612903,3.225806,4.838710,1.612903,6.451613,1.612903,17.741935,3.225806,1.612903,0.999995
3,nonDRNA,362,39968.9808,-0.141713,0.334254,0.281768,0.356354,187,74,57,...,4.419890,4.143646,2.209945,3.314917,4.696133,5.524862,6.906077,0.276243,2.486188,0.771930
4,RNA,270,30399.2510,-0.314074,0.300000,0.248148,0.355556,138,62,28,...,4.074074,5.555556,4.444444,7.407407,3.703704,5.185185,9.629630,1.851852,4.444444,1.499999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8790,nonDRNA,297,31947.6303,0.172727,0.333333,0.252525,0.400673,162,68,32,...,3.030303,3.703704,4.713805,3.367003,5.050505,7.407407,7.070707,0.000000,1.683502,1.093750
8791,nonDRNA,213,23640.7253,-0.172770,0.352113,0.309859,0.375587,109,45,29,...,2.816901,7.042254,1.877934,0.938967,8.920188,4.225352,7.511737,1.877934,2.347418,1.034482
8792,nonDRNA,227,24041.5607,0.062996,0.180617,0.383260,0.427313,117,86,13,...,7.929515,6.607930,2.643172,2.202643,10.132159,9.251101,11.453744,1.762115,6.167401,0.846153
8793,nonDRNA,980,109294.1143,-0.382041,0.333673,0.291837,0.343878,424,307,145,...,4.183673,4.285714,6.734694,3.877551,10.612245,5.918367,5.204082,0.408163,2.653061,0.717241


In [6]:
protien[protien['protien_seq'].apply(lambda x: 'Z' in x)]

,protien_seq,class


In [7]:
clean_protiens2 = clean_data(protien, substitute=True)

In [8]:
bio_features2 = extract_features(clean_protiens)

In [9]:
bio_features2

,class,protien_len,mol_weighht,isoelectric_point,Helix,Turn,Sheet,type1,type2,type3,...,N,P,Q,R,S,T,V,W,Y,pos_neg_rat
0,nonDRNA,155,17596.7878,-0.630968,0.341935,0.238710,0.329032,73,37,19,...,4.516129,3.225806,5.161290,6.451613,2.580645,6.451613,5.806452,3.870968,3.225806,1.368420
1,nonDRNA,336,37260.7955,-0.201190,0.366071,0.303571,0.366071,168,75,43,...,5.952381,3.273810,1.190476,2.976190,6.845238,4.166667,6.547619,0.297619,3.571429,1.162790
2,nonDRNA,62,6764.1710,1.579032,0.322581,0.209677,0.580645,48,10,2,...,1.612903,3.225806,4.838710,1.612903,6.451613,1.612903,17.741935,3.225806,1.612903,0.999995
3,nonDRNA,362,39968.9808,-0.141713,0.334254,0.281768,0.356354,187,74,57,...,4.419890,4.143646,2.209945,3.314917,4.696133,5.524862,6.906077,0.276243,2.486188,0.771930
4,RNA,270,30399.2510,-0.314074,0.300000,0.248148,0.355556,138,62,28,...,4.074074,5.555556,4.444444,7.407407,3.703704,5.185185,9.629630,1.851852,4.444444,1.499999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8790,nonDRNA,297,31947.6303,0.172727,0.333333,0.252525,0.400673,162,68,32,...,3.030303,3.703704,4.713805,3.367003,5.050505,7.407407,7.070707,0.000000,1.683502,1.093750
8791,nonDRNA,213,23640.7253,-0.172770,0.352113,0.309859,0.375587,109,45,29,...,2.816901,7.042254,1.877934,0.938967,8.920188,4.225352,7.511737,1.877934,2.347418,1.034482
8792,nonDRNA,227,24041.5607,0.062996,0.180617,0.383260,0.427313,117,86,13,...,7.929515,6.607930,2.643172,2.202643,10.132159,9.251101,11.453744,1.762115,6.167401,0.846153
8793,nonDRNA,980,109294.1143,-0.382041,0.333673,0.291837,0.343878,424,307,145,...,4.183673,4.285714,6.734694,3.877551,10.612245,5.918367,5.204082,0.408163,2.653061,0.717241
